In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook, Workbook
from datetime import datetime
import os
import time
import pandas as pd
import re
import shutil
# Função para inserir a data atual no campo "txtInicio"
def inserir_data_atual(navegador):
    # Obtendo a data atual no formato DD/MM/AAAA
    data_atual = datetime.now().strftime("%d/%m/%Y")
    
    # Esperando o campo "txtInicio" estar presente e visível
    campo_data_inicio = WebDriverWait(navegador, 10).until(
        EC.visibility_of_element_located((By.NAME, "txtInicio"))
    )
    
    # Limpando o campo e inserindo a data atual
    campo_data_inicio.clear()
    campo_data_inicio.send_keys(data_atual)

# Inicializando o navegador Chrome
navegador = webdriver.Chrome()
navegador.get("")

# Aguardando até que os campos de login estejam presentes
WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.NAME, "txtLogin")))

# Preenchendo os campos de login e senha
navegador.find_element(By.NAME, "txtLogin").send_keys("")
navegador.find_element(By.NAME, "txtSenha").send_keys("")

# Selecionando a opção no campo selectInstituicao
navegador.find_element(By.NAME, "selectInstituicao").send_keys("")

# Clicando no botão de login usando espera explícita
botao_entrar = WebDriverWait(navegador, 10).until(EC.element_to_be_clickable((By.ID, "btnLogar")))
botao_entrar.click()

# Aguardando e clicando no botão de menu
botao_menu = WebDriverWait(navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/nav/ul[1]/li[1]/a')))
botao_menu.click()

# Aguardando e clicando no botão de relatório
botao_relatorio = WebDriverWait(navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gerenciarRelatorio"]')))
botao_relatorio.click()

# Selecionando o relatório
selecionar_relatorio = WebDriverWait(navegador, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="sidebarMenu"]/ul/li[16]/ul/li[1]/a')))
selecionar_relatorio.click()

# Inserindo a data atual no campo "txtInicio"
inserir_data_atual(navegador)

# Esperando o campo "txtFim" e preenchendo com a data desejada
campo_data_fim = WebDriverWait(navegador, 10).until(
    EC.visibility_of_element_located((By.NAME, "txtFim"))
)
campo_data_fim.clear()
campo_data_fim.send_keys("31/12/2024")

# Aguardando até que o dropdown "Horario_length" esteja disponível
mostrar_todos_dropdown = WebDriverWait(navegador, 20).until(
    EC.presence_of_element_located((By.NAME, "Horario_length"))
)

# Usando Select para selecionar a opção "Todos"
select_mostrar_todos = Select(mostrar_todos_dropdown)
select_mostrar_todos.select_by_visible_text("Todos")

# Esperando até que o botão Excel esteja clicável
botao_excel = WebDriverWait(navegador, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="Horario_wrapper"]/div[1]/button[1]'))
)

# Rolando até o botão Excel para garantir que esteja visível
navegador.execute_script("arguments[0].scrollIntoView(true);", botao_excel)

# Tentando clicar no botão usando JavaScript se o clique normal falhar
try:
    botao_excel.click()
except:
    navegador.execute_script("arguments[0].click();", botao_excel)


import os
import re
import shutil
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill, Border, Side

# Função para identificar o último arquivo baixado com uma extensão específica
def encontrar_ultimo_arquivo_baixado(diretorio_downloads, extensao=".xlsx"):
    # Listando os arquivos no diretório de downloads com a extensão correta
    arquivos = [f for f in os.listdir(diretorio_downloads) if f.endswith(extensao)]
    
    # Verificando se existem arquivos com a extensão desejada
    if not arquivos:
        raise FileNotFoundError(f"Nenhum arquivo com a extensão {extensao} encontrado em {diretorio_downloads}")
    
    # Verificando o caminho completo de cada arquivo e retornando o mais recente
    caminhos_arquivos = [os.path.join(diretorio_downloads, arquivo) for arquivo in arquivos]
    ultimo_arquivo = max(caminhos_arquivos, key=os.path.getmtime)
    return ultimo_arquivo

# Função para mover o arquivo baixado para outro diretório
def mover_arquivo(caminho_arquivo, destino):
    if not os.path.exists(destino):
        os.makedirs(destino)  # Cria o diretório de destino, caso ele não exista
    novo_caminho = os.path.join(destino, os.path.basename(caminho_arquivo))
    shutil.move(caminho_arquivo, novo_caminho)
    return novo_caminho

# Caminho do diretório de downloads (modifique conforme seu sistema)
diretorio_downloads = os.path.expanduser("~/Downloads")

# Diretório de destino para onde o arquivo será movido
diretorio_destino = os.path.expanduser(r"")

# Encontrando o último arquivo baixado com extensão .xlsx
try:
    NOME_ARQUIVO = encontrar_ultimo_arquivo_baixado(diretorio_downloads, extensao=".xlsx")
except FileNotFoundError as e:
    print(e)
    NOME_ARQUIVO = None

# Se encontrou o arquivo, movê-lo para o diretório de destino e processá-lo
if NOME_ARQUIVO:
    novo_caminho_arquivo = mover_arquivo(NOME_ARQUIVO, diretorio_destino)

    # Se o arquivo foi movido com sucesso, processá-lo
    if os.path.exists(novo_caminho_arquivo):
        print(f"Arquivo movido para: {novo_caminho_arquivo}")

        # Função para carregar a planilha
        def carregar_planilha(caminho_arquivo):
            return load_workbook(filename=caminho_arquivo)

        # Função para limpar o nome da aba
        def limpar_nome_aba(nome):
            if nome is None:
                return "Sem_Nome"
            return re.sub(r'[\\/*?:[\]]', '', nome)[:31]

        # Função para criar ou obter uma aba existente
        def obter_ou_criar_aba(workbook, nome_aba):
            if nome_aba in workbook.sheetnames:
                return workbook[nome_aba]
            nova_aba = workbook.create_sheet(title=nome_aba)
            adicionar_titulos(nova_aba)
            return nova_aba

        # Função para adicionar os títulos das colunas
        def adicionar_titulos(aba):
            aba['A1'] = "Data"
            aba['B1'] = "Horário de Início"
            aba['C1'] = "Horário de Fim"
            aba['D1'] = "Turma"

        # Função para preencher a aba com os dados
        def preencher_aba(aba, dados):
            linha = len(aba['A']) + 1  # Encontrar a próxima linha disponível
            aba[f"A{linha}"] = dados['data']
            aba[f"B{linha}"] = dados['inicio']
            aba[f"C{linha}"] = dados['fim']
            aba[f"D{linha}"] = dados['turma']

        # Função para extrair dados de uma linha da planilha principal
        def extrair_dados_linha(sheet, linha):
            return {
                'data': sheet[f"A{linha}"].value,
                'inicio': sheet[f"C{linha}"].value,
                'fim': sheet[f"D{linha}"].value,
                'turma': sheet[f"E{linha}"].value
            }

        # Função principal para processar a planilha
        def processar_planilha(nome_arquivo):
            workbook = carregar_planilha(nome_arquivo)
            sheet_principal = workbook.active

            # Dicionário para manter o controle das novas planilhas por instrutor
            planilhas_instrutores = {}

            # Processar os dados de cada linha
            for linha in range(2, len(sheet_principal['F']) + 1):
                nome_instrutor = limpar_nome_aba(sheet_principal[f'F{linha}'].value)

                if nome_instrutor not in planilhas_instrutores:
                    planilhas_instrutores[nome_instrutor] = Workbook()
                    selecionaSheetInstrutor = planilhas_instrutores[nome_instrutor].active
                    selecionaSheetInstrutor.title = "Instrutores"

                    # Preenche os títulos das colunas na nova planilha do instrutor
                    adicionar_titulos(selecionaSheetInstrutor)

                aba_instrutor = planilhas_instrutores[nome_instrutor]
                dados_instrutor = extrair_dados_linha(sheet_principal, linha)
                preencher_aba(aba_instrutor['Instrutores'], dados_instrutor)

            # Salvar cada nova planilha de instrutor
            for nome_instrutor, workbook_instrutor in planilhas_instrutores.items():
                caminho_nova_planilha = os.path.join(diretorio_destino, f"{nome_instrutor}.xlsx")
                workbook_instrutor.save(filename=caminho_nova_planilha)
                print(f"Nova planilha criada para {nome_instrutor}: {caminho_nova_planilha}")

            # Salvar as alterações na planilha principal
            workbook.save(filename=nome_arquivo)
            os.startfile(nome_arquivo)

        # Executar o processamento da planilha
        processar_planilha(novo_caminho_arquivo)
    else:
        print("Erro: O arquivo não foi movido corretamente.")
else:
    print("Nenhum arquivo encontrado para processar.")

import win32com.client as win32 # Importa a biblioteca para automação do Outlook

# Inicializa a aplicação Outlook
outlook = win32.Dispatch('outlook.application')

# Define o caminho do arquivo Excel que contém os dados dos emails
nome_arquivo = r"caminho do arquivo"

# Carregar a planilha
planilha_aberta = load_workbook(filename=nome_arquivo)
# Seleciona a aba 'Dados', que contém as informações necessárias para o envio de emails
sheet_selecionada = planilha_aberta['Dados']
# Loop que percorre todas as linhas da planilha, começando da linha 2 (assumindo que a linha 1 são cabeçalhos)
for linha in range(2, len(sheet_selecionada['A']) + 1):
    # Obtém os valores de cada coluna da linha atual
    nome = sheet_selecionada['A%s' % linha].value
    nomeCompleto = sheet_selecionada['B%s' % linha].value
    email = sheet_selecionada['C%s' % linha].value
    # Cria um novo email no Outlook
    emailOutlook = outlook.CreateItem(0)
    # Define o destinatário do email
    emailOutlook.To = email
    # Define o assunto do email, que inclui o nome completo da pessoa
    emailOutlook.Subject = "Horário " + nomeCompleto
    # Define o corpo do email em formato HTML
    emailOutlook.HTMLBody = f"""
    <p>Boa noite <b>{nome}</b>.</p>
    <p>Segue seu horário referente a segunda quinzena do mês.</p>
    <p>Atenciosamente.</p>
    """
    # Define o caminho do anexo, que é um arquivo Excel com o nome da pessoa
    anexoEmail = "Caminho do arquivo" + nomeCompleto + ".xlsx"
    
    # Verifica se o arquivo existe antes de anexar
    if os.path.exists(anexoEmail):
       # Se o arquivo existir, ele será anexado ao email   
        emailOutlook.Attachments.Add(anexoEmail)
    else:
         # Se o arquivo não for encontrado, uma mensagem de erro será exibida
        print(f"Erro: O arquivo {anexoEmail} não foi encontrado.")

    emailOutlook.Send()   # .Send() envia o email; .save() poderia ser usado para apenas salvar